In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# basic import
import os
import os.path as op
import sys
sys.path.insert(0, op.dirname(os.getcwd()))

# python libs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [18, 8]

# tk libs
from lib.objs.tkpaths import Site
from lib.data_fetcher.STORMS import Download_NOAA_WMO
from lib.tcyclone import Extract_Circle
from lib.plotting.storms import WorldMap_Storms


# --------------------------------------
# Site paths and parameters
site = Site('KWAJALEIN')

DB = site.pc.DB                        # common database
ST = site.pc.site                      # site database
PR = site.params                       # site parameters

# input files
p_wvs_parts = ST.wvs.partitions_p1
p_hist_tcs = DB.tcs.noaa_fix

# output files
p_hist_tcs = DB.tcs.noaa
p_hist_tcs_fix = DB.tcs.noaa_fix
p_tcs_circle_hist = ST.tcs.circle_hist

# wave point lon, lat and radius for TCs selection
pnt_lon = float(PR.WAVES.point_longitude)
pnt_lat = float(PR.WAVES.point_latitude)
r2 = float(PR.TCS.r2)   # smaller one

In [4]:
# --------------------------------------
# Historical TCs  

# load WMO TCs from netCDF file
xds_wmo = xr.open_dataset(p_hist_tcs)

# set lon to 0-360
lon_wmo = xds_wmo.lon_wmo.values[:]
lon_wmo[np.where(lon_wmo<0)] = lon_wmo[np.where(lon_wmo<0)]+360
xds_wmo['lon_wmo'].values[:] = lon_wmo

# modify some variable names
xds_wmo.rename(
    {'lon_wmo':'lon',
     'lat_wmo':'lat',
     'time_wmo':'dates',
     'pres_wmo':'pressure',
    }, inplace=True)

# store fixed wmo file
xds_wmo.to_netcdf(p_hist_tcs_fix)

print xds_wmo
print ''


# Select TCs that crosses a circular area 
print(
'\nExtracting Historical TCs from WMO database...\n \
Lon = {0:.2f}º , Lat = {1:.2f}º, R2  = {2:6.2f}º'.format(
    pnt_lon, pnt_lat, r2)
)

xds_TCs_r, xds_inside = Extract_Circle(
    xds_wmo, pnt_lon, pnt_lat, r2)


# store data
xds_TCs_r.to_netcdf(p_tcs_circle_hist)
print('\nHistorical TCs (inside circle) stored at:\n{0}'.format(p_tcs_circle_hist))

print ''
print xds_TCs_r

# TODO: ADD PLOT (trazas huracanes seleccionados sobre mapa mundo)
#WorldMap_Storms(xds_hurr_r)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in less
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The inplace argument has been deprecated and will be removed in xarray 0.12.0.


<xarray.Dataset>
Dimensions:          (center: 26, storm: 7367, time: 137)
Dimensions without coordinates: center, storm, time
Data variables:
    storm_sn         (storm) |S13 ...
    name             (storm) |S57 ...
    numObs           (storm) int16 ...
    season           (storm) int16 ...
    track_type       (storm) int8 ...
    genesis_basin    (storm) int8 ...
    num_basins       (storm) int8 ...
    basin            (storm, time) float32 ...
    wind_avg_period  (center) float32 ...
    source           (center) |S10 ...
    dates            (storm, time) datetime64[ns] ...
    lat              (storm, time) float32 ...
    lon              (storm, time) float32 79.799995 78.9 73.2 ... nan nan nan
    alt              (storm, time) float32 ...
    wind_wmo         (storm, time) float32 ...
    pressure         (storm, time) float32 ...
    sub_basin        (storm, time) float32 ...
    nature_wmo       (storm, time) float32 ...
    source_wmo       (storm, time) float32 ...